In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/housesalesprediction/kc_house_data.csv")


In [ ]:
df.info()

In [ ]:
df.describe().transpose()

# Features

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(df["price"])

In [ ]:
sns.countplot(df["bedrooms"])

In [ ]:
sns.barplot(x="bedrooms", y="price", data=df)

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x="sqft_living",y="price",data=df)

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x="sqft_basement", y="price", data=df)

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x="long", y="price", data=df)

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x="lat", y="price", data=df)

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x="long", y="lat",
                data=df, hue="price",
                palette="RdYlBu", edgecolor=None)

In [ ]:
sns.boxplot(x="waterfront", y="price", data=df)

In [ ]:
sns.boxplot(x="floors", y="price", data=df)

In [ ]:
sns.boxplot(x="condition", y="price", data=df)

In [ ]:
sns.boxplot(x="grade", y="price", data=df)

In [ ]:
sns.boxplot(x="view", y="price", data=df)

In [ ]:
df.sort_values("price", ascending=False).head(50)

# Remove houses worth more than 3 million and create a model for houses up to 3 million. Discard 45 most expensive houses out of 21613.

In [ ]:
df = df.sort_values("price", ascending=False).iloc[45:]

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x="long", y="lat",
                data=df, hue="price",
                palette="RdYlBu", edgecolor=None)

In [ ]:
df.head()

In [ ]:
df = df.drop("id", axis=1)

In [ ]:
df["date"] = pd.to_datetime(df["date"])

In [ ]:
df["year"] = df["date"].apply(lambda date:date.year)

In [ ]:
df["month"] = df["date"].apply(lambda date:date.month)

In [ ]:
df.groupby("year").mean()["price"]

In [ ]:
sns.barplot(x="month", y="price", data=df)

In [ ]:
df = df.drop("date", axis=1)

In [ ]:
df.head()

In [ ]:
df["zipcode"].value_counts()

Requires domain expertise for feature engineering.

In [ ]:
df = df.drop("zipcode", axis=1)

In [ ]:
df["yr_renovated"].value_counts()

In [ ]:
df.head()

In [ ]:
features = ["price", "bedrooms", "sqft_living", "sqft_lot", "floors", "waterfront",
            "view", "condition", "grade", "sqft_above", "sqft_basement", "yr_built",
            "yr_renovated", "lat", "long", "sqft_living15", "sqft_lot15", "year", "month"]
mask = np.zeros_like(df[features].corr(), dtype=np.bool) 
mask[np.triu_indices_from(mask)] = True 

f, ax = plt.subplots(figsize=(16, 12))
plt.title('Pearson Correlation Matrix',fontsize=25)

sns.heatmap(df[features].corr(),linewidths=0.25,vmax=0.7,square=True,cmap="BuGn",
            linecolor='w',annot=True,annot_kws={"size":8},mask=mask,cbar_kws={"shrink": .9});


# Train Test Split

In [ ]:
X = df.drop("price", axis=1)
y = df["price"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)

60% train, 20% validation and 20% test

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

# Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_val = scaler.transform(X_val)

In [ ]:
X_test = scaler.transform(X_test)

# Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()

model.add(Dense(19, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))

model.add(Dense(1))

model.compile(optimizer="adam", loss="mse")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
#early_stop = EarlyStopping(monitor="val_loss", patience=1000)

In [ ]:
check_point = ModelCheckpoint("best_model.h5", monitor="val_loss", verbose=0, save_best_only=True)

In [ ]:
model.fit(x=X_train, y=y_train.values,
          validation_data=(X_val, y_val.values),
          batch_size=32, epochs=10000, 
          callbacks=[check_point], verbose=0)

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

In [ ]:
from keras.models import load_model

In [ ]:
saved_model = load_model('best_model.h5')

In [ ]:
predictions = saved_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
r2 = r2_score(y_test, predictions)

In [ ]:
avg = np.mean(y_test)

In [ ]:
print("Average house price in test set: {}".format(avg))
print("RMSE: {}".format(rmse))
print("R-squared score: {}".format(r2))